# Autogen with GPT-4o

## ConversableAgent

In [8]:
import os
import json
from autogen import ConversableAgent, AssistantAgent, GroupChat, GroupChatManager
from dotenv import load_dotenv, find_dotenv
from promptflow.tracing import start_trace
from opentelemetry import trace


load_dotenv(find_dotenv())

endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
key = os.environ["AZURE_OPENAI_KEY"]
api = os.environ["API_VERSION"]
deployment = os.environ["CHAT_COMPLETIONS_DEPLOYMENT_NAME"]

# This has a cache_seed
llm_config = {"model": deployment, "base_url": endpoint, "api_key": key, "api_type": "azure", "api_version": api,
               "temperature": 0.8, "max_tokens": 4096, "top_p": 0.95, "cache_seed": 13}

# THIS HAS NO CACHE SEED SET
# llm_config = {"model": deployment, "base_url": endpoint, "api_key": key, "api_type": "azure", "api_version": api,
#                "temperature": 0.8, "max_tokens": 4096, "top_p": 0.95, "cache_seed": None}

### Automatic Cache feature

You will need to delete the .cache folder or set a new cache_seed
[LLM Caching with Autogen](https://microsoft.github.io/autogen/docs/topics/llm-caching/#:~:text=To%20change%20the%20cache%20directory%2C%20set%20cache_path_root%3A%20with,on%20by%20default%20with%20cache_seed%20set%20to%2041.)

In [9]:
# baseline = ConversableAgent(
#     name="Baseline Comparison Bot",
#     system_message="You are an AI assistant that helps people find information.",
#     llm_config=llm_config,
#     code_execution_config=False,  # Turn off code execution, by default it is off.
#     function_map=None,  # No registered functions, by default it is None.
#     human_input_mode="NEVER",  # Never ask for human input. human_input_mode: Literal['ALWAYS', 'NEVER', 'TERMINATE'] = "TERMINATE",
# )

# baselinereply = baseline.generate_reply(messages=[{"content": "Write a persuasive essay about why Dr. Dog is the greatest band ever. Use markdown format. The essay should start with an outline and aim to be close to 2,000 words.",
#                                               "role": "user"}])
# print(baselinereply)

## Set Up Multi-Agent Conversation

> NOTE: the name= parameter must not include spaces

In [10]:
# Agent for collecting a topic from a user
topic_agent = ConversableAgent(
    name="TopicCollectionAgent",
    system_message="You are part of a persuasive essay writing team and you present the topic for the essay to the group. ",
    llm_config=llm_config,
    description="Collect a topic from a user for a persuasive essay.",
    human_input_mode="NEVER",
)

# Agent for creating an outline
outline_agent = ConversableAgent(
    name="OutlineAgent",
    system_message="You are part of a persuasive essay writing team and you create a detailed outline for the persuasive essay based on the supplied topic. ",
    llm_config=llm_config,
    description="Create an outline for a persuasive essay based on a topic.",
    human_input_mode="NEVER",
)

# Agent for creating an outline
writing_agent = ConversableAgent(
    name="WritingAgent",
    system_message="You are part of a persuasive essay writing team and you act as the main author. You should begin writing the essay based on the supplied outline. ",
    llm_config=llm_config,
    description="Write a persuasive essay based on a supplied outline.",
    human_input_mode="NEVER",
)

# Agent for creating an outline
review_agent = ConversableAgent(
    name="ReviewAgent",
    system_message="You are part of a persuasive essay writing team and you act as the reviewer and critic. Keep a keen eye on the structure, grammar, and overall persuasiveness of the essay. Return the final product in markdown format.",
    llm_config=llm_config,
    description="Review a persuasive essay and make necessary edits.",
    human_input_mode="NEVER",
)

### Create GroupChat, GroupChatManager, and .initiate_chat

In [11]:
# Create the GroupChat object
writing_group_chat = GroupChat(
    agents=[topic_agent, outline_agent, writing_agent, review_agent],
    messages=[],
    max_round=10, # 10 is default
    send_introductions=True, # send_introductions: send a round of introductions at the start of the group chat,
     # so agents know who they can speak to (default: False)
    speaker_selection_method='auto', # 'auto' is default ['auto', 'manual', 'random', 'round_robin']
)

# Create the GroupChatManager object
writing_group_manager = GroupChatManager(
    groupchat=writing_group_chat,
    llm_config=llm_config,
)

### Add in Prompt Flow Tracing

In [12]:
from promptflow.tracing import start_trace

# start a trace session, and print a url for user to check trace
# traces will be collected into below collection name
start_trace(collection="autogen-writinggrouptest")

Starting prompt flow service...
You can stop the prompt flow service with the following command:'pf service stop'.
Alternatively, if no requests are made within 1 hours, it will automatically stop.


In [13]:
tracer = trace.get_tracer("my_tracer")
# Create a root span
with tracer.start_as_current_span("autogen") as span:
    # This begins the actual chat and provides the group with the topic as listed in the message.
    message="Why Dr. Dog is the greatest band ever."
    # chat_result = topic_agent.initiate_chat(
    topic_agent.initiate_chat(
    writing_group_manager,
    message=message,
    summary_method="reflection_with_llm",
    )
    span.set_attribute("custom", "custom attribute value")
    # recommend to store inputs and outputs as events
    span.add_event(
        "promptflow.function.inputs", {"payload": json.dumps(dict(message=message))}
    )
    span.add_event(
        "promptflow.function.output", {"payload": json.dumps(topic_agent.last_message())}
    )
# type exit to terminate the chat

TopicCollectionAgent (to chat_manager):

Why Dr. Dog is the greatest band ever.

--------------------------------------------------------------------------------
OutlineAgent (to chat_manager):

Great! The topic for our persuasive essay is "Why Dr. Dog is the greatest band ever." Now, I'll hand it over to OutlineAgent to create a detailed outline for this persuasive essay.

--------------------------------------------------------------------------------
OutlineAgent (to chat_manager):

Sure, let's create a detailed outline for the persuasive essay on why Dr. Dog is the greatest band ever.

**I. Introduction**
   A. Hook: A compelling quote from one of Dr. Dog's songs or a powerful statement about their influence.
   B. Background information: Brief introduction to Dr. Dog, including formation, genre, and key members.
   C. Thesis statement: Dr. Dog is the greatest band ever because of their unique sound, compelling lyrics, and dedicated fan base.

**II. Unique Sound**
   A. Fusion of g

You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=autogen-writinggrouptest&uiTraceId=0x8aedf72899d709470b8b5beef9ba1937


NameError: name 'message' is not defined

In [ ]:
from opentelemetry import trace
import json


tracer = trace.get_tracer("my_tracer")
# Create a root span
with tracer.start_as_current_span("autogen") as span:
    message = "Find a latest paper about gpt-4 on arxiv and find its potential applications in software."
    user_proxy.initiate_chat(
        manager,
        message=message,
        clear_history=True,
    )
    span.set_attribute("custom", "custom attribute value")
    # recommend to store inputs and outputs as events
    span.add_event(
        "promptflow.function.inputs", {"payload": json.dumps(dict(message=message))}
    )
    span.add_event(
        "promptflow.function.output", {"payload": json.dumps(user_proxy.last_message())}
    )
# type exit to terminate the chat